# IMAP Data Access API

The IMAP SDC provides a Data Access API to **query** and **download** data. There are three ways to use the API:

1. The `imap-data-access` command-line utility
2. The `imap_data_access` python package
3. Calls to the REST API directly

This notebook provies instructions and examples of how to use the command-line utility and python package. *For information on how to use the REST API directly, see https://imap-processing.readthedocs.io/en/latest/data-access/index.html*

## Data Directory

Before using the API, it is important to note that the folder structure for data files within the IMAP SDC is rigidly defined, so the data access will mimic that structure to make sure all data is stored in the same heirarchical structure as the SDC. This will enable seamless transition between a user's local system and the SDC.

The directory structure is as follows:

```
<IMAP_DATA_DIR>/
  imap/
    <instrument>/
      <data_level>/
        <year>/
          <month>/
            <filename>
```

for example:

```
/data/
  imap/
    swe/
      l0/
        2024/
          01/
            imap_swe_l0_sci_20240105_v001.pkts
```

By default, the `<IMAP_DATA_DIR>` is the user's current working directory. However, this can be changed by setting the `IMAP_DATA_DIR` environment variable:

In [ ]:
!mkdir -p /Users/mabo8927/imap_data/
%env IMAP_DATA_DIR=/Users/mabo8927/imap_data
!echo $IMAP_DATA_DIR

## Installation

The API can be installed via `pip`:

In [ ]:
!pip install imap-data-access
!imap-data-access --version

We leave it as an exercise to the user to install the package into their software environment of choice.

## Command Line Utility

The following section shows examples of how to use the API via the `imap-data-access` command line utility.

In [ ]:
# Show help documentation
!imap-data-access -h

#### Querying

In [ ]:
# Show query help documentation
!imap-data-access query -h

In [ ]:
# Query with a single parameter
!imap-data-access query --instrument swe

In [ ]:
# Query with multiple parameters
!imap-data-access query --instrument swe --data-level l1b --start-date 20240510 --end-date 20240510

In [ ]:
# Change the output format
!imap-data-access query --instrument swe --data-level l1b --start-date 20240510 --end-date 20240510 --output-format json

In [ ]:
# Query based on a filename
!imap-data-access query --filename imap_swe_l1b_sci_20240510_v012.cdf

#### Downloading

In [ ]:
# Show download help documentation
!imap-data-access download -h

##### Science Files

In [ ]:
# Download a science file
!imap-data-access download imap_swe_l1b_sci_20240510_v012.cdf

# Check to see that the file exists
!ls -lt $IMAP_DATA_DIR/imap/swe/l1b/2024/05/imap_swe_l1b_sci_20240510_v012.cdf

##### Ancillary Files

In [ ]:
# Download an ancillary file
!imap-data-access download imap_mag_calibration_20240229_v001.cdf

# Check to see that the file exists
!ls -lt $IMAP_DATA_DIR/imap/ancillary/mag/imap_mag_calibration_20240229_v001.cdf

## Python Package
The following section shows examples of how to use the API via the `imap_data_access` Python package.

In [ ]:
# import the package
import imap_data_access

The `<IMAP_DATA_DIR>` can be set via the `config` dictionary:

In [ ]:
from pathlib import Path

imap_data_access.config["DATA_DIR"] = Path.home() / "imap_data"

#### Querying

In [ ]:
results = imap_data_access.query(instrument="swe", data_level="l0")
for result in results:
    print(result)

#### Download a file

In [ ]:
file_path = imap_data_access.download("imap_swe_l0_raw_20241212_v000.pkts")
print(file_path)

#### Bonus: validate/construct a file path

In [ ]:
science_file = imap_data_access.ScienceFilePath("imap_mag_l0_raw_20250502_v001.pkts")
print(science_file.start_date)
print(science_file.descriptor)

filepath = science_file.construct_path()
print(filepath)

## Future work

The SDC continues to develop the API for the greater needs of the IMAP community. We currently have plans to add the following features (subject to change):

- Add support for querying ancillary files
- Add support for querying and downloading SPICE files
- Add query parameter for querying based on repointing
- Add `today` as a query parameter to return list of files that were acquired in the last 24 hours
- Add `days_ago=N` as a query parameter to return a list of file that were acquired in the last `N` days
- Create a production instance of the API